In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import pandas as pd

In [3]:

# Fonction pour scraper les produits d'une catégorie donnée (homme ou femme)
def scrape_shoes_category(url, gender, max_pages, max_items=None):
    all_products = []
    
    for page in range(1, max_pages + 1):  # Boucler sur chaque page
        # Modifier l'URL pour inclure le numéro de page
        page_url = f"{url}&page={page}"
        response = requests.get(page_url)
        
        if response.status_code != 200:
            print(f"Erreur de connexion à {page_url}")
            continue
        
        # Parser la page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Trouver tous les éléments contenant les produits
        product_containers = soup.find_all('li', class_='musheji_name')  # Utilisation de la classe spécifique
        
        for container in product_containers:
            # Récupérer le nom du produit à partir de l'attribut 'title' de la balise <a>
            name = container.find('a')['title']  # Le nom du produit
            # Récupérer le prix à partir de la classe 'productSpecialPrice'
            price_tag = container.find_next('span', class_='productSpecialPrice')
            if price_tag:
                price = price_tag.text.strip()  # Nettoyer le prix
            else:
                price = "N/A"  # Si le prix n'est pas disponible

            all_products.append({'name': name, 'price': price, 'gender': gender})
    
    return all_products

# URL des pages de chaussures pour hommes et femmes
women_url = 'https://shopping.louboutin2024.com/index.php?main_page=index&cPath=2'
men_url = 'https://shopping.louboutin2024.com/index.php?main_page=index&cPath=155'
bebes_url = 'https://shopping.louboutin2024.com/index.php?main_page=index&cPath=232'
filles_url = 'https://shopping.louboutin2024.com/index.php?main_page=index&cPath=236'
cadeaux_homme_url = 'https://shopping.louboutin2024.com/index.php?main_page=index&cPath=175'
cadeaux_femme_url = 'https://shopping.louboutin2024.com/index.php?main_page=index&cPath=55'

# Scraper les données pour chaque catégorie
women_shoes = scrape_shoes_category(women_url, 'Femme', max_pages=16)
men_shoes = scrape_shoes_category(men_url, 'Homme', max_pages=7)
bebes_shoes = scrape_shoes_category(bebes_url, 'Bébé', max_pages=2)  
filles_shoes = scrape_shoes_category(filles_url, 'Fille', max_pages=2)  
cadeaux_homme = scrape_shoes_category(cadeaux_homme_url, 'Cadeau Homme', max_pages=3, max_items=47)
cadeaux_femme = scrape_shoes_category(cadeaux_femme_url, 'Cadeau Femme', max_pages=2, max_items=15)


In [4]:
all_products = women_shoes + men_shoes + bebes_shoes + filles_shoes + cadeaux_homme + cadeaux_femme

# Créer un DataFrame Pandas avec les données
df = pd.DataFrame(all_products)
df.head(7)

,name,price,gender
0,So CL Mule Mules 55 mm - Cuir nappa - Bianco -...,€59.00,Femme
1,So CL Mule Mules 55 mm - Cuir de veau - Noir -...,€55.00,Femme
2,So CL Mule Mules 55 mm - Cuir de veau - Leche ...,€55.00,Femme
3,So CL Mule Mules 55 mm - Cuir de veau - Cuoio ...,€57.00,Femme
4,CL Mule Mules - Cuir de veau - Leche - Femme,€57.00,Femme
5,CL Mule Mules - Cuir de veau - Noir - Femme,€57.00,Femme
6,CL Mule Mules - Cuir nappa - Bianco - Femme,€56.00,Femme


In [5]:
# Séparer les colonnes en 'collection', 'product_name', et 'sexe'
df[['collection', 'product_name']] = df['name'].str.split(' - ', expand=True, n=1)  # Diviser une première fois en deux parties
df['sexe'] = df['name'].str.rsplit(' - ', n=1).str[-1]  # Récupérer le dernier élément après le dernier tiret


In [6]:
df.isnull().sum()

name            0
price           0
gender          0
collection      0
product_name    1
sexe            0
dtype: int64

In [7]:
df.describe()

,name,price,gender,collection,product_name,sexe
count,488,488,488,488,487,488
unique,476,8,6,233,264,30
top,Lou Babe Ballerines - Crêpe satin - Loubi - Bébés,€55.00,Femme,Lou Babe Ballerines,Cuir de veau - Noir - Femme,Femme
freq,2,101,252,12,26,259


In [8]:
# Nettoyer les prix (enlever le symbole € et convertir en float)
df['price'] = df['price'].replace('€', '', regex=True)
df['price'] = df['price'].replace(',', '.', regex=True)

# Convertir les prix en float, les erreurs seront transformées en NaN
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [9]:
# Calcul du prix moyen
average_price = df['price'].mean()

# Afficher le prix moyen
print(f"Prix moyen des produits : {average_price:.2f} €")

Prix moyen des produits : 55.63 €


In [10]:
# Calcul du nombre de produits par genre
product_count_by_gender = df['gender'].value_counts()
print(f"Nombre de produits par genre :\n{product_count_by_gender}")

Nombre de produits par genre :
gender
Femme           252
Homme           101
Cadeau Homme     48
Cadeau Femme     32
Fille            31
Bébé             24
Name: count, dtype: int64
